In [1]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StringType,IntegerType,ArrayType,StructField
from pyspark.sql.types import StructType as struct

import pyspark.sql.functions as func
from enum import Enum
from pyspark.sql.functions import udf
from pyspark.sql.types import Row
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
sc=SparkContext.getOrCreate()

In [2]:
ORACLE_DRIVER_PATH = "C:\Program Files\Java\ojdbc5.jar"                                            
Oracle_CONNECTION_URL ="jdbc:oracle:thin:TPA_DEV_APP01/password@CTSC00847233301:1521:orcl"
sqlContext = SQLContext(sc)
ora_tmp=spark.read.format('jdbc').options(
        url=Oracle_CONNECTION_URL,
        dbtable="ASPOLICY",
        driver="oracle.jdbc.driver.OracleDriver"
        ).load()

In [3]:
ora_tmp.createOrReplaceTempView("people")
asd=spark.sql("SELECT POLICYGUID,POLICYNUMBER FROM people")

In [4]:
#asd.show()

In [5]:
tab2=spark.read.format('jdbc').options(
        url=Oracle_CONNECTION_URL,
        dbtable="ASPOLICYFIELD",
        driver="oracle.jdbc.driver.OracleDriver"
        ).load()

In [6]:
ta2=asd.join(tab2,['POLICYGUID'],'outer')
ta2.createOrReplaceTempView("dat1")

In [7]:
orgtab2=spark.sql("SELECT  POLICYGUID,POLICYNUMBER,FIELDNAME,FIELDTYPECODE,DATEVALUE,TEXTVALUE,INTVALUE,FLOATVALUE FROM dat1")

In [8]:
# ab= orgtab2.groupBy("POLICYNUMBER","POLICYGUID").\
#   agg(func.collect_list(func.create_map(func.col("FIELDNAME"),
#    func.col("FIELDTYPECODE"))).alias("VALUES"))
#def separate

In [9]:
orgtab2=orgtab2.withColumn("DATEVALUE", orgtab2["DATEVALUE"].cast(StringType()))
#orgtab2=orgtab2.withColumn("FIELDTYPECODE", orgtab2["FIELDTYPECODE"].cast(IntegerType()))
orgtab2=orgtab2.withColumn("INTVALUE", orgtab2["INTVALUE"].cast(StringType()))
orgtab2=orgtab2.withColumn("FLOATVALUE", orgtab2["FLOATVALUE"].cast(StringType()))


def my_udf(df):
    if (df.FIELDTYPECODE is'1'):
        return df.DATEVALUE
    if(df.FIELDTYPECODE is'2'):
        return df.TEXTVALUE
    if(df.FIELDTYPECODE is'3'):
        return df.INTVALUE
    if(df.FIELDTYPECODE is'4'):
        return df.FLOATVALUE

#i=0    
#udf_object=udf(my_udf,ArrayType( lambda x : struct(i for i in range(0,492),StringType(),True ))
#orgtab2.withColumn("Values", (udf_object((struct([orgtab2[x] for x in orgtab2.columns])))))
#orgtab2.withColumn("Values",( lambda x :func.concat_ws("",x) ,my_udf(orgtab2)))

root
 |-- POLICYGUID: string (nullable = true)
 |-- POLICYNUMBER: string (nullable = true)
 |-- FIELDNAME: string (nullable = true)
 |-- FIELDTYPECODE: string (nullable = true)
 |-- DATEVALUE: timestamp (nullable = true)
 |-- TEXTVALUE: string (nullable = true)
 |-- INTVALUE: decimal(10,0) (nullable = true)
 |-- FLOATVALUE: decimal(38,10) (nullable = true)

root
 |-- POLICYGUID: string (nullable = true)
 |-- POLICYNUMBER: string (nullable = true)
 |-- FIELDNAME: string (nullable = true)
 |-- FIELDTYPECODE: string (nullable = true)
 |-- DATEVALUE: string (nullable = true)
 |-- TEXTVALUE: string (nullable = true)
 |-- INTVALUE: string (nullable = true)
 |-- FLOATVALUE: string (nullable = true)



In [10]:

def func(FIELDTYPECODE, DATEVALUE ,TEXTVALUE,INTVALUE,FLOATVALUE):
    if  (FIELDTYPECODE == '01'):
        return DATEVALUE
    elif(FIELDTYPECODE == '02'):
        return TEXTVALUE
    elif(FIELDTYPECODE == '03'):
        return INTVALUE
    elif(FIELDTYPECODE == '04'):
        return FLOATVALUE
    
func_udf = udf(func, StringType())
orgtab2= orgtab2.withColumn("Vals",func_udf(orgtab2['FIELDTYPECODE'],orgtab2['DATEVALUE'],orgtab2['TEXTVALUE'],orgtab2['INTVALUE'],orgtab2['FLOATVALUE']))

#s=orgtab2.rdd
#orgtab2.select((1)).collect()[1][1].show()
#aq=ad=0
#for i in orgtab2:
#    print orgtab2.select().collect()[3]
#    aq=aq+1
#i=0
#for x in ab:
#    x.select(('POLICYNUMBER')).collect()[i]["POLICYNUMBER"].show()
#    i=i+1
#orgtab2.groupby("POLICYGUID").count().show()

In [11]:
orgtab2.createOrReplaceTempView("dat12")

In [18]:
ab= orgtab2.filter(orgtab2.FIELDNAME == "AppReceivedDate")

In [19]:
ab.createOrReplaceTempView("dat123")
ab1= spark.sql("SELECT POLICYNUMBER,FIELDNAME,Vals FROM dat123")

In [20]:
ab1.write.format('json').save('/path/AppReceivedDate.json')